# Assignment 2

In this assignment you will implement ResNet18.
Read the comments carefully and insert your code where you see: <br><br><b>##### START OF YOUR CODE #####</b><br><br><b>##### END OF YOUR CODE #####</b><br><br>or for the inline codes you will see<br><br><b>##### INSERT YOUR CODE HERE #####</b>

### The architecture of ResNet-18 is shown in the table.
First, we will define a convolutional block with skip connection. Then, create the model using these blocks.<br><br>
<img src="https://www.researchgate.net/profile/Paolo-Napoletano/publication/322476121/figure/tbl1/AS:668726449946625@1536448218498/ResNet-18-Architecture.png" width="500" alt="ResNet18 Architecture">

<br><sup>Image ref: Napoletano, Paolo, et al. ‘Anomaly Detection in Nanofibrous Materials by CNN-Based Self-Similarity’. Sensors (Basel, Switzerland), vol. 18, 01 2018, https://doi.org10.3390/s18010209.</sup>

#### I. ConvBlock
<img src="https://www.researchgate.net/publication/334301817/figure/fig3/AS:778452965801986@1562609058538/Residual-block-of-ResNet18-with-a-1-1-convolutional-mapping-based-residual-unit-and.png"><br>
ResNet consists of convolutional (a) and identity (b) blocks. For ResNet-18 we will only use convolutional blocks. In this step you will write a class for convolutional block. The arguments will be:

* ch_in: input channels
* ch_out: output channels
* s: strides
* act: activation function

The options for activation function are "relu", "leaky_relu" and "gelu".
<br><br>
<sup>Image ref: Owais, Muhammad, et al. ‘Artificial Intelligence-Based Classification of Multiple Gastrointestinal Diseases Using Endoscopy Videos for Clinical Diagnosis’. Journal of Clinical Medicine, vol. 8, 07 2019, p. 986, https://doi.org10.3390/jcm8070986.</sup>

In [55]:
import torch
from torch import nn
from torch.nn import functional as F

class ConvBlock(nn.Module):
    def __init__(self, ch_in, ch_out, s, act):
      super(ConvBlock,self).__init__()
      # Initialize layers
      ##### START OF YOUR CODE #####
      self.ch_in = ch_in
      self.ch_out =ch_out
      self.s = s
      self.bn = nn.BatchNorm2d(ch_out)
      self.act = act

      if self.act.lower() == "relu":
        self.act = nn.ReLU()
      elif act.lower() == "leaky_relu":
        self.act = nn.Leaky_ReLU()
      elif act.lower() == "gelu":
        self.act = nn.GELU()

      #convolutional layers of one block
      if self.s == 1:
        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size = (1,1), stride = s, padding = 0)
        self.conv2 = nn.Conv2d(ch_in, ch_out, kernel_size = (3,3), stride = s, padding = 1)
        self.conv3 = nn.Conv2d(ch_out, ch_out, kernel_size = (3,3), stride = s, padding = 1)    
      else:
        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size = (1,1), stride = s, padding = 0)
        self.conv2 = nn.Conv2d(ch_in, ch_out, kernel_size = (3,3), stride = s, padding = 1)
        self.conv3 = nn.Conv2d(ch_out, ch_out, kernel_size = (3,3), stride = s-1, padding = 1)
      
      ##### END OF YOUR CODE #####

    def forward(self, X):
      ##### START OF YOUR CODE #####      
      x = self.conv1(X)
      x = self.bn(x)
      
      y = self.conv2(X)
      y = self.bn(y)
      y = self.act(y)
      y = self.conv3(y)
      y = self.bn(y)

      y += x # skip connection
      X = self.act(y)
      ##### END OF YOUR CODE #####
      return X

#### II. ResNet18 class
Use the ConvBlock class to create ResNet18.
* Add batch normalization and activation function after the first conv layer as well.
* Examine the output sizes in the table above and use paddings and strides where needed.
* Pytorch doesn't have a global average pooling layer. Instead you should reshape the image as (B, C, W*H) and calculate the mean of the last dimension without keeping the dims. It will result in a tensor of (B, C)
* Add drop-out layer after average pooling.
* Fully connected layer should be 512 x 1 as we have only 2 classes and we will use sigmoid function as the final activation layer.

<img src="https://www.researchgate.net/profile/Paolo-Napoletano/publication/322476121/figure/tbl1/AS:668726449946625@1536448218498/ResNet-18-Architecture.png" width="500" alt="ResNet18 Architecture">

In [56]:
class ResNet18(nn.Module):
    def __init__(self, act, drop_rate):
      super(ResNet18, self).__init__()
      # Initialize layers
      ##### START OF YOUR CODE #####
      self.drop_rate = drop_rate
      self.drop_out = nn.Dropout2d(drop_rate)
      self.max_pool = nn.MaxPool2d(kernel_size = (3,3), stride = (2,2), padding = (1,1))
      self.avg_pool = nn.AvgPool2d(kernel_size = (7,7))
      self.fully_connected = nn.Linear(512,1)
      self.sigmoid = nn.Sigmoid()
      self.bn = nn.BatchNorm2d(64)  
      self.flatten = nn.Flatten()
      self.act = act 

      if self.act.lower() == "relu":
        self.act = nn.ReLU()
      elif act.lower() == "leaky_relu":
        self.act = nn.Leaky_ReLU()
      elif act.lower() == "gelu":
        self.act = nn.GELU()

      #convolutional blocks
      self.conv1 = nn.Conv2d(1, 64, kernel_size = (7,7), stride = (2,2), padding = (3,3))
      
      self.conv2_x = nn.Sequential(ConvBlock(64, 64, 1, act),
                                   ConvBlock(64, 64, 1, act))
      
      self.conv3_x = nn.Sequential(ConvBlock(64, 128, 2, act),
                                   ConvBlock(128, 128, 1, act))
      
      self.conv4_x = nn.Sequential(ConvBlock(128, 256, 2, act),
                                   ConvBlock(256, 256, 1, act))
      
      self.conv5_x = nn.Sequential(ConvBlock(256, 512, 2, act),
                                   ConvBlock(512, 512, 1, act))
          
      ##### END OF YOUR CODE #####

    def forward(self, X):
      ##### START OF YOUR CODE #####
      X = self.conv1(X)
      X = self.bn(X)
      X = self.act(X)
      X = self.max_pool(X)
      X = self.conv2_x(X)
      X = self.conv3_x(X)
      X = self.conv4_x(X)
      X = self.conv5_x(X)
      X = self.avg_pool(X) 
      X = self.drop_out(X)
      X = self.flatten(X)     
      X = self.fully_connected(X)
      X = self.sigmoid(X)
      ##### END OF YOUR CODE #####
      return X

In [57]:
from torchsummary import summary

# Print the summary of model
#device = torch.device('cuda') --> For GPU
model = ResNet18("relu", .5) #.to(device) -->For GPU
summary(model, (1, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           3,200
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]           4,160
       BatchNorm2d-6           [-1, 64, 64, 64]             128
            Conv2d-7           [-1, 64, 64, 64]          36,928
       BatchNorm2d-8           [-1, 64, 64, 64]             128
              ReLU-9           [-1, 64, 64, 64]               0
           Conv2d-10           [-1, 64, 64, 64]          36,928
      BatchNorm2d-11           [-1, 64, 64, 64]             128
             ReLU-12           [-1, 64, 64, 64]               0
        ConvBlock-13           [-1, 64, 64, 64]               0
           Conv2d-14           [-1, 64,